In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
TEMPLATE = """
Interprete the text and evaluate the text.
sentiment: is the text in a positive, neutral or negative sentiment? Sentiment is required.
subject: What subject is the text about? Use exactly one word. Use 'None' if no subject was provided.
price: How much did the customer pay? Use 'None' if no price was provided.

Format the output as JSON with the following keys:
sentiment
subject
price

text: {input}
"""

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains.llm import LLMChain

In [4]:
#Initialize LLM Model
llm = ChatOpenAI(model="gpt-4o-mini", temperature = 0)

#Initializing and declaring prompt template
promptTemplate = ChatPromptTemplate.from_template(template=TEMPLATE)

#Attach LLM and Prompt
chain = LLMChain(llm=llm, prompt=promptTemplate)

#Perform Query using Chain object
chain.invoke(input="I ordered Chicken Pizza from Dominos. It was cold and I didnt like it!")



/var/folders/cc/_kcvjw313pl6kbf8c3bdtnyc0000gn/T/ipykernel_70477/4288336830.py:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=promptTemplate)


{'input': 'I ordered Chicken Pizza from Dominos. It was cold and I didnt like it!',
 'text': '```json\n{\n  "sentiment": "negative",\n  "subject": "Pizza",\n  "price": "None"\n}\n```'}

# ResponseSchema

In [5]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
#if type not declared in ResponseSchema, default type is string

response_schemas = [
    ResponseSchema(name="sentiment", description="is the text in a positive, neutral or negative sentiment? Sentiment is required."),
    ResponseSchema(name="subject", description="What subject is the text about? Use exactly one word."),
    ResponseSchema(name="price", description="How much did the customer pay? Use None if no price was provided.", type="float")
]
print(response_schemas)

[ResponseSchema(name='sentiment', description='is the text in a positive, neutral or negative sentiment? Sentiment is required.', type='string'), ResponseSchema(name='subject', description='What subject is the text about? Use exactly one word.', type='string'), ResponseSchema(name='price', description='How much did the customer pay? Use None if no price was provided.', type='float')]


In [6]:
parser = StructuredOutputParser.from_response_schemas(response_schemas)
formatInstructions = parser.get_format_instructions()
print(formatInstructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"sentiment": string  // is the text in a positive, neutral or negative sentiment? Sentiment is required.
	"subject": string  // What subject is the text about? Use exactly one word.
	"price": float  // How much did the customer pay? Use None if no price was provided.
}
```


In [7]:
# Create prompt template
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate

prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "Interprete the text and evaluate the text. "
            "sentiment: is the text in a positive, neutral or negative sentiment? "
            "subject: What subject is the text about? Use exactly one word. "
            "Just return the JSON, do not add ANYTHING, NO INTERPRETATION! "
            "text: {input}\n"
            "{format_instructions}\n"
        )
    ],
    input_variables=["input"],
    partial_variables={"format_instructions": formatInstructions}
)

In [8]:
#Attach LLM and Prompt
chain1 = LLMChain(llm=llm, prompt=prompt)

#Perform Query using Chain object
output = chain1.invoke(input="I ordered Chicken Pizza from Dominos. It was cold and I didnt like it!")



In [9]:
print(output['text'])

```json
{
	"sentiment": "negative",
	"subject": "Pizza",
	"price": null
}
```
